# Neural Networks with TensorFlow and Keras

Deep neural networks는 수십 혹은 수백개 layer가 될 수도 있다. "deep"이라는 단어가 여기서 유례했다고 할 수 있다. 이전 notebook에서 본바와 같이 weight 행렬만 가지고도 이런 deep networks들 중에 하나를 생성할 수 있다. 하지만 일반적으로 복잡하고 구현하기 어려워진다. TensorFlow는 **Keras** 라는 멋진 API를 가지고 있으며 효율적으로 대규모 neural networks를 만들 수 있는 멋진 방법을 제공한다.

## Import Resources

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [ ]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [ ]:
print('Using:')
print('\t\u2022 TensorFlow version:', tf.__version__)
print('\t\u2022 tf.keras version:', tf.keras.__version__)
print('\t\u2022 Running on GPU' if tf.test.is_gpu_available() else '\t\u2022 GPU device not found. Running on CPU')

## Load the Dataset

이제 대규모 network를 만들꺼고 이 network는 어려운 문제를 해결하는데 image 내에 문자를 식별하는 것이다. 여기서 MNIST dataset을 사용하는데 이 데이터는 greyscale의 손으로 쓴 숫자로 구성되어 있다. 각 image는 28x28 pixels이고 아래와 같은 샘플을 볼 수 있을 것이다.

<img src='assets/mnist.png'>

neural network를 생성하는 우리의 목표는 이런 images들 중에 하나를 가져와서 이 image에 있는 숫자를 추측하는 것이다.

먼저 dataset을 얻고 우리의 Neural Network을 train시키고 test해서 사용할 예정이다. [`tensorflow_datasets`](https://www.tensorflow.org/datasets) 패키지를 사용하여 dataset을 얻는다. TensorFlow Dataset은 TensorFlow와 함께 사용하기 위해 준비된 datasets의 저장소이다. TensorFlow Datasets은 다양한 서로 다른 tasks를 위한 머신러닝 모델을 train시키기 위해서 광범위한 datasets들ㅇ르 가지고 있다.(문자에서 비디오까지) dataset의 전체 목록은 TensorFlow Datasets인 [TensorFlow Datasets Catalog](https://www.tensorflow.org/datasets/catalog/overview#all_datasets)를 확인해보자.


아래 코드는 MNIST dataset을 로드한다.

In [ ]:
# training data를 로드 (Load training data)
training_set, dataset_info = tfds.load('mnist', split = 'train', as_supervised = True, with_info = True)

## Inspect the Dataset

training data가 `training_set`로 로드되고 dataset 정보는 `dataset_info`로 로드되었다. `dataset_info`로부터 classes들의 총 개수와 training set에 있는 images의 전체 수를 얻어오자.

In [ ]:
num_classes = dataset_info.features['label'].num_classes
print('There are {:,} classes in our dataset'.format(num_classes))

num_training_examples = dataset_info.splits['train'].num_examples
print('\nThere are {:,} images in the training set'.format(num_training_examples))

`training_set` 을 iterator로 사용해서 dataset에 loop를 위해서 다음과 같이 구문을 사용한다:

```python
for image, label in training_set:
    ## do things with images and labels
```

이미지와 labels의 shape와 dtype을 출력해보자. `.take(1)` method를 사용해서 dataset에 있는 하나의 element만 선택한다. dataset은 images들로 구성되어 있으므로 `.take(1)` method는 하나의 image만 선택할 것이다.

In [ ]:
for image, label in training_set.take(1):
    print('The images in the training set have:')
    print('\u2022 dtype:', image.dtype) 
    print('\u2022 shape:', image.shape)
  
    print('\nThe labels of the images have:')
    print('\u2022 dtype:', label.dtype) 

dataset에서 images들은 `shape = (28, 28, 1)` 와 `dtype = uint8` 의 tensors이다. `unit8` 은 8-bit unsigned integer이고 0~255범위의 정수값을 가질 수 있다. 반면에 images의 label은 `dtype = int64` 의 tensors인데 이말은 64-bit signed integers라는 것을 뜻한다. 이제 dataset에서 하나의 image는 어떤 형태인지 알아보자. images를 그리기 위해서 먼저 TensorFlow tensors로부터 NumPy ndarrays로 `.numpy()` method를 사용해서 변환해야한다. images는 `shape = (28, 28, 1)`를 가지므로 `.squeeze()` method를 사용해서 images를 `shape = (28, 28)` 로 reshape한다. `.squeeze()` method는 ndarray의 shape으로부터 single-dimensional entries를 제거한다.

In [ ]:
for image, label in training_set.take(1):
    image = image.numpy().squeeze()
    label = label.numpy()
    
# Plot the image
plt.imshow(image, cmap = plt.cm.binary)
plt.colorbar()
plt.show()

print('The label of this image is:', label)

## Create Pipeline

본바와 같이 images의 pixel 값들은 `[0, 255]` 범위에 있다. images를 정규화시키고 training set으로 pipeline을 생성한다. neural network에 공급할 수 있다. 이미지를 정규화시키기 위해서 pixel 값을 255로 나눈다. 따라서 먼저 image의 `dtype` 을 `tf.cast` function을 사용해서 `uint8` 에서 `float32` 로 변경한다.(32-bit single-precision floating-point numbers)

In [ ]:
def normalize(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

batch_size = 64

training_batches = training_set.cache().shuffle(num_training_examples//4).batch(batch_size).map(normalize).prefetch(1)

`64` batch size로 pipeline을 생성했다는 것을 알고 dataset을 섞는다. batch size는 한번의 iteration에서의 images의 개수고 network로 전달된다. 이를 *batch* 라고 부른다. `shuffle` 변환은 network에 feed되기전에 random하게 dataset의 엘리머트를 섞는다.

비록 이런 많은 변환은 교환이 가능하며 변환의 순서는 성능에 영향을 미친다. 이런 변환과 성능에 대해서 더 상세한 정보는 아래 링크를 참고하자 :

* [Pipeline Performance](https://www.tensorflow.org/beta/guide/data_performance)


* [Transformations](https://www.tensorflow.org/api_docs/python/tf/data/Dataset)

이제 `training_batches` 를 가지고 이를 살펴보자. :

In [ ]:
for image_batch, label_batch in training_batches.take(1):
    print('The images in each batch have:')
    print('\u2022 dtype:', image_batch.dtype) 
    print('\u2022 shape:', image_batch.shape)
  
    print('\nThere are a total of {} image labels in this batch:'.format(label_batch.numpy().size))
    print(label_batch.numpy())

이제 batches 중에 하나로부터 단일 image를 구하는 방법을 살펴보자.

In [ ]:
# images의 단일 batch를 받아서 squeezing으로 color dimension을 제거 (Take a single batch of images, and remove the color dimension by squeezing it)
for image_batch, label_batch in training_batches.take(1):
    images = image_batch.numpy().squeeze()
    labels = label_batch.numpy()

# Plot the image
plt.imshow(images[0], cmap = plt.cm.binary)
plt.colorbar()
plt.show()

print('The label of this image is:', labels[0])

## Build a Simple Neural Network

먼저 이 dataset에 weight 행렬과 행렬 곱셈을 이용해서 간단한 network를 생성한다. 이전 notebook에서 했던 것과 동일하다. 다음으로 TensorFlow와 Keras를 사용해서 어떻게 하는지 보도록하자. 이렇게 하면 network 구조를 정의하는 간편하고 강력한 방법을 제공한다.

우리가 보왔던 networks들은 *fully-connected* 혹은 *dense* networks라고 불린다. 하나의 layer에서 각 unit은 다음 layer에 있는 각 unit에 연결된다. 완전히 연결된 networks에서 각 layer에 대한 입력은 반드시 1차원 vecotr여야 한다.(2D tensor로 stack이 가능) 하지만 여기서 images는 28 $\times$ 28 2D tensors 이고 이를 1D vectors로 변환할 필요가 있다. sizes에 대해서 생각하자 shape `(64, 28, 28, 1)` 의 images의 batch를 shape of `(64, 784)`으로 변환할 필요가 있고 784는 28 x 28이다. 이를 일반적으로 *flattening* 이라고 하고 2D images를 1D vectors로 flattened시키는 것이다.

이전 notebook에서 하나의 output unit을 가지는 network를 만들었다. 여기서 10개 output units를 필요하고 각 숫자(digit)에 대해서 하나이기 때문이다. 여기서 network이 이미지에 보이는 숫자를 추정하게 하기 위해서 해당 image가 어떤 숫자나 어느 class인지 확률을 계산한다. 해당 숫자(class)에 대한 분산 확률 분산 나와서 해당 이미지가 어떤 숫자인거 같은지를 말해준다. 이 말은 10개 classes(digit)에 대해서 10개 output units를 필요로 한다. network output을 확률 분산으로 변환하는 방법을 보여준다.

> **연습문제:** 위에서 우리가 생성한  images의 batch인 `images`를 flatten시킨다. 다음으로 784 input units, 256 hidden units, 10개 output units 에 대해서 weights와 biases를 위한 random tensors를 사용해서 간단한 network를 생성한다. 이제 hidden layer에서 units을 위한 sigmoid activation function을 사용한다. activation 없이 outpout liayer를 남겨둔다. 다음으로 확률 분산을 주는 것을 추가한다. **HINT:** images의 batch를 flatten시키기 위해서 [`tf.reshape()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/reshape) 를 사용할 수 있다.

In [ ]:
## Solution

output = 

# output의 shape을 출력 (Print the shape of the output. It should be (64,10))
print('The output has shape:', output.shape)

이 network에는 10개 output을 가진다. 하나의 image를 network으로 전달하고 classes에 대해서 확률 분포가 나오며 이것은 해당 이미지가 어느 class와 가장 유사한지를 말해준다. 아래처럼 나타난다 :
<img src='assets/image_distribution.png' width=500px>

여기서 각 class에 대한 확률은 대략 동일하다. 이는 network이 train되지 않았다는 것을 의마하며 아직ㅈ 어떤 data를 보지 않아서 각 class에 대해서 동일한 확률을 가지는 동일 형태의 분산을 반환한다.

이 확률 분포를 계산하는데 [**softmax** function](https://en.wikipedia.org/wiki/Softmax_function) 가 많이 사용된다.
수식표현은 아래와 같다

$$
\Large \sigma(x_i) = \cfrac{e^{x_i}}{\sum_k^K{e^{x_k}}}
$$

이것이 하는 일은 각 input $x_i$ 를 0과 1사이로 squish하고 값들을 적절한 확률 분포로 정규화시켜서 확률의 합이 1이 되도록 한다.

> **연습문제:** `softmax` 함수를 구현해보자. 이 함수는 softmax 계산을 수행하고 batch에 있는 각 예제에 대해서 확률 분포를 반환한다. 이를 수행할때 shapes에 대해서 주의를 기울여야 한다. tensor `a` 는 shape `(64, 10)` 를 가지고 tensor `b` 는 shape `(64,)` 가지는 경우 `a/b`를 수행하면 error가 발생한다. 왜냐하면 TensorFlow는 컬럼들에 대해서 division을 수행하려고(broadcasting이라고 불리는) 하지만 size가 맞지 않기 때문이다. 이에 대한 생각 방식은 다음과 같다 : 64개 예제 각각에 대해서, 하나의 값으로만 나누기를 원하는 경우, 분모에 합이 온다. 따라서 `(64, 1)` shape을 가지기 위해서 `b` 가 필요하다. 이런 방식으로 TensorFlow는 `a` 의 각 행에 10개 값으로 나누고 `b` 의 각 행에 하나의 값으로 나눈다. 합을 어떻게 가져올지도 주의해야한다. `tf.reduce_sum()` 에 `axis` 키워드를 정의할 필요가 있다. `axis=0` 설정은 `axis=1` 가 컬럼들에 대해서 합을 가지는 동안 행에 대한 합을 가진다. output tensor가 제대로된 shape `(64,1)`을 가지는 것을 확신하기 위해서는 `tf.reduce_sum()` 에서 `keepdims` 키워드를 사용이 필요하다.

In [ ]:
## Solution


# Apply softmax to the output
probabilities = softmax(output)

# Print the shape of the probabilities. Should be (64, 10).
print('The probabilities have shape:', probabilities.shape, '\n')


# The sum of probabilities for each of the 64 images should be 1
sum_all_prob = tf.reduce_sum(probabilities, axis = 1).numpy()

# Print the sum of the probabilities for each image.
for i, prob_sum in enumerate(sum_all_prob):
    print('Sum of probabilities for Image {}: {:.1f}'.format(i+1, prob_sum))

## Building Neural Networks with TensorFlow and Keras

Keras는 neural networks를 구성하고 trian시키기 위한 하이레벨 API이다. `tf.keras` 는 Keras API의 TensorFlow의 구현이다. Keras에서 deep learning 모델들은 **layers** 라는 설정 가능한 빌딩 블록들을 연결해서 구성한다. 가장 일반적인 타입의 모델은 **Sequential** 모델이라고 불리는 layers의 stack이다. 이 모델을 sequential이라고 부르는 이유는 각 layer에 operations을 통해서 tensor가 순차적으로 통과되는 것을 허용하기 때문이다. TensorFlow에서 sequential 모델은 `tf.keras.Sequential` 로 구현한다.

아래 cell에서 Keras sequential 모델을 사용해서 동일한 완전 연결 neural network을 만든다. 이전 섹션에서 만들어 보았다. 여기서의 sequential 모델은 3개 layers로 구성되어 있다 :

* **Input Layer:** `tf.keras.layers.Flatten` — This layer flattens the images by transforming a 2d-array of 28 $\times$ 28 pixels, to a 1d-array of 784 pixels (28 $\times$ 28 = 784). The first layer in a Sequential model needs to know the shape of the input tensors to the model. Since, this is our first layer, we need to specify the shape of our input tensors using the `input_shape` argument. The `input_shape` is specified using a tuple that contains the size of our images and the number of color channels. It is important to note that we don't have to include the batch size in the tuple. The tuple can have integers or `None` entries, where `None` entries indicate that any positive integer may be expected.

* **Hidden Layer:** `tf.keras.layers.Dense` — A fully-connected (also known as densely connected) layer. For this layer we need to specify the number of neurons (or nodes) we want to use and the activation function. Note that we don't have to specify the shape of the input tensor to this layer, since Keras performs automatic shape inference for all layers except for the first layer. In this particular case, we are going to use `256` neurons with a `sigmoid` activation fucntion. 

* **Output Layer:** `tf.keras.layers.Dense` — A fully-connected layer with 10 neurons and a *softmax* activation function. The output values will represent the probability that the image is a particular digit. The sum of all the 10 nodes values is 1.

In [ ]:
model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape = (28,28,1)),
        tf.keras.layers.Dense(256, activation = 'sigmoid'),
        tf.keras.layers.Dense(10, activation = 'softmax')
])

model.summary()

### Your Turn to Build a Neural Network

<img src="assets/mlp_mnist.png" width=600px>

> **연습문제:** 784개 input units, hidden layer는 128개 units 그리고 ReLU activation를 가지며 hidden layer는 64개 units과 ReLU activation을 가지며 마지막으로 output layer는 10개 units과 softmax activation function을 가지고 있다. `activation = 'relu'` 설정으로 ReLU activation function을 사용할 수 있다.

In [ ]:
## Solution
my_model_1 = 

my_model_1.summary()

## Activation Functions

지금까지 softmax activation을 살펴봤다. 하지만 일반적으로 어떤 함수든 activation function으로 사용될 수 있다. 어떤 network가 비선형 함수를 approximate하기 위한 유일한 요구사항은 activation function은 반드시 비선형이어야만 한다. 일반 activation functions의 몇 가지 추가 예제로 :  Tanh (hyperbolic tangent), and ReLU (rectified linear unit) 가 있다.

<img src="assets/activation.png" width=700px>

실제로 ReLU function은 hidden layers에 대해서 거의 독보적으로 activation funtion으로 사용된다.

## Looking at the Weights and Biases

Keras는 자동으로 weights와 biases를 초기화해준다. weights와 biases는 model에 정의한 각 layer에 부착된 tensors이다. `get_weights` method를 사용해서 model로부터 모든 weights와 biases를 얻을 수 있다. `get_weights` method는 NumPy arrays로서 model에서 모든 weight와 bias tensor 목록을 반환한다.

In [ ]:
model_weights_biases = model.get_weights()

print(type(model_weights_biases))

In [ ]:
print('\nThere are {:,} NumPy ndarrays in our list\n'.format(len(model_weights_biases)))

print(model_weights_biases)

`get_layer` method를 사용해서 특정 layer에 대한 weights와 biases를 얻을 수 있다. 이 경우 전에 했던 것과 `index` argument를 사용해서 먼저 layer를 지정하고 `get_weights` method를 적용한다. 예제로 sequential model의 첫번째 layer의 weights와 biases를 얻기 위해서 다음을 사용한다:

```python
weights = model.get_layer(index=0).get_weights()[0]
biases = model.get_layer(index=0).get_weights()[1]

```

주의 : model의 첫번째 layer를 가져오기 위해서 사용한 `index=0`은 `tf.keras.layers.Flatten` 이다. 이 layer는 우리 input을 flatten만 하고 weights나 biases는 가지지 않는다. 따라서 이 경우 `index=0`를 가지는 layer는 weights나 biases가 없다. `get_weights()`은 빈 목록 (`[]`)을 반환할 것이기 때문에 `get_weights()[0]` 호출은 error를 생성하게 된다. 

대안으로 model의 layers의 목록을 가져오기 위해서 `layers` method를 사용할 수도 있다. layers에 대한 loop를 돌리고  `get_weights()` 호출하기 전에 weights를 가지고 있는지를 검사한다. 예제를 한 번 보자.:

In [ ]:
# Dislay the layers in our model
model.layers

In [ ]:
for i, layer in enumerate(model.layers):
    
    if len(layer.get_weights()) > 0:
        w = layer.get_weights()[0]
        b = layer.get_weights()[1]
        
        print('\nLayer {}: {}\n'.format(i, layer.name))
        print('\u2022 Weights:\n', w)
        print('\n\u2022 Biases:\n', b)
        print('\nThis layer has a total of {:,} weights and {:,} biases'.format(w.size, b.size))
        print('\n------------------------')
    
    else:
        print('\nLayer {}: {}\n'.format(i, layer.name))
        print('This layer has no weights or biases.')
        print('\n------------------------')

보는바와 같이 기본적으로 모든 biases는 0으로 초기화된다.

반면에 기본적으로 weights들은 Glorot uniform initializer를 통해서 초기화된다. 초기화는 \[-`limit`, `limit`\] 내에 uniform distribution으로부터 samples를 그리며 여기서 `limit` 은 `sqrt(6 / (fan_in + fan_out))` 이고 여기서 `fan_in`은 weight tensor에 있는 input units의 개수이고 `fan_out`은 weight tensor에 있는 output units의 개수이다.

Keras에서 weights와 biases에 대한 기본 초기화 method를 변경할 수 있다. 유효한 초기화에 대해서 좀더 알고자 한다면 아래 링크는 확인해보자.:

* [Available initializers](https://keras.io/initializers/)

* [Dense Layer](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense)

## Make Predictions

가지고 있는 model로 images batch에 prediction을 하기 위해서 `.predict(image_batch)` method를 사용한다. 이 method는 batch에 images를 가져와서 network에 넣어줘서 전방으로 통과하면서 처리된다. batch에 있는 각 image에 대해서 추정확률을 가지는  shape `(batch_size, num_classes)`의 NumPy ndarray를 출력한다.

batch에 대해서 64개 images를 가지고 dataset은 10 classes (*i.e.* `num_classes = 10`) 를 가지므로 model은 shape `(64,10)`의 array를 출력할 것이다. 이 array에 있는 행은 images에 대해서 예측 확률을 가진다. 결과적으로 첫번째 행은 batch에 있는 첫번째 image에 대해서 추정 확률을 가진다. 2번째 행은 batch에 있는 2번째 image에 대한 추정확률을 가진다. 3번째 행은  batch에 있는 3번째 image에 대한 추정확률을 가진다. 이 경우 추정확률은 10개 값으로 구성되어 즉 class당 하나의 확률이 된다. 따라서 batch에 있는 64개 images 각각에 대해서 10개 확률을 가지게 된다. 

batch에 있는 첫번째 image에 대한 model의 추정 확률을 그려보자.

In [ ]:
for image_batch, label_batch in training_batches.take(1):
    ps = model.predict(image_batch)
    first_image = image_batch.numpy().squeeze()[0]


fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
ax1.imshow(first_image, cmap = plt.cm.binary)
ax1.axis('off')
ax2.barh(np.arange(10), ps[0])
ax2.set_aspect(0.1)
ax2.set_yticks(np.arange(10))
ax2.set_yticklabels(np.arange(10))
ax2.set_title('Class Probability')
ax2.set_xlim(0, 1.1)
plt.tight_layout()

위에서 본바와 같이 model은 모든 숫자에 대해서 대략 동일한 확률을 부여한다. 이 말은 우리 network은 기본적으로 image에 있는 해당 숫자가 무엇인지 모른다는 것을 의미한다. 이는 아직 우리 model을 train시키지 않았기 떄문이고 모든 weight는 random이다!

## Subclassing with TensorFlow and Keras

`tf.keras.Sequential` model은 layers의 간단한 stack으로 임의의 model을 생성하는데 사용할 수는 없다. 운좋게 `tf.keras`는 `tf.keras.Model` 을 subclass 및 forward pass를 정의해서 완전한 커스텀 model을 생성하는데 필요한 유연성을 제공한다.

다음 예제에서 subclassed `tf.keras.Model`를 사용해서 위에서 만들었던 784 inputs, 256 hidden units, 10 output units의 동일한 nueral network을 만든다. 전과 같이 hidden layer에서 units에 대한 ReLu activation function을 사용하고 output neurons에 대해서 Softmax activation function를 이용한다.

In [ ]:
class Network(tf.keras.Model):
    def __init__(self, num_classes = 2):
        super().__init__()
        self.num_classes = num_classes
    
        # Define layers 
        self.input_layer = tf.keras.layers.Flatten()
        self.hidden_layer = tf.keras.layers.Dense(256, activation = 'relu')
        self.output_layer = tf.keras.layers.Dense(self.num_classes, activation = 'softmax')
    
    # Define forward Pass   
    def call(self, input_tensor):
        x = self.input_layer(input_tensor)
        x = self.hidden_layer(x)
        x = self.output_layer(x)
    
        return x 

하나씩 상세히 알아보자.

```python
class Network(tf.keras.Model):
```

`tf.keras.Model`로부터 상속받는다. `super().__init__()`와 결합되어 많은 유용한 methods와 attributes를 제공하는 class를 생성한다. network를 위한 class를 생성할때 `tf.keras.Model`에서 상속받아야만 한다. 하지만 class 이름은 어떤 것이든 상관없다.

다음으로 `__init__` method 내부에서 network의 layers를 생성하고 class instance의 attributes로 설정한다. output layer에 neurons의 개수를 할당을 `__init__` method 에서 수행하면 이는  `num_classes` 인자를 통해서 가능하다. 기본값은 2이다.

```python
self.input = tf.keras.layers.Flatten()
```

앞에서 말한바와 같이 첫번째 layer는 input image를 flatten한다. 이 layer에  `self.input` 이름을 붙인다. 이후에 이 layer를 참조하는데 이 이름을 사용할 것이다. 여러분의 layer에 어떤 이름을 무엇으로 하는지는 중요하지 않으면 어떤 이름이든 가능하다.

```python
self.hidden = tf.keras.layers.Dense(256, activation = 'relu')
```

2번째 layer는 256 neurons와 ReLu activation function을 가지는 완전히 연결된(dense) layer이다. 이 layer에 이름을 `self.hidden`으로 한다. 이 이름으로 이 layer에 대한 참조로 사용한다.

```python
self.output = tf.keras.layers.Dense(self.num_classes, activation = 'softmax')
```

3번째와 마지막 layer(output layer)도 `self.num_classes` neurons와 softmax activation function으로 fully-connected (dense) layer이다. 기본적으로 output의 개수는 2가 되지만 여러분의 dataset의 output classes의 개수에 따라서 다른 정수값을 사용할 수 있다.

다음으로 `call` method에서 forward pass를 정의한다.

```python
def call(self, input_tensor):
```

`tf.keras.Model`로 생성된 TensorFlow models은 반드시 정의된 `call` method를 가져야만 한다. `call` method에서 `input_tensor`를 가지고 `__init__` method에서 정의한 layers로 이것ㅇ르 pass시킨다.

```python
x = self.input(input_tensor)
x = self.hidden(x)
x = self.output(x)
```

`input_tensor`는 각 layer를 통과하고 `x`로 재할당한다. `input_tensor`는 `input` layer에 들어가고 다음으로 `hidden` layer에 들어가고 마지막으로 `output` layer에 들어간다. `__init__` method에서 layers 정의한 순서는 중요하지 않다. 하지만 `call` method에서는 순서가 중요하다. `__init__` method에서 각 layer에 대해서 부여한 이름을 참조한다. 이 이름은 임의로 정한것이라는 것을 명심하자.

이제 model class를 정의하고 `model` object를 생성할 수 있다. `Network` class에서 input tensor의 shape을 지정하지 않는다. 이 경우 weights와 biases만 초기화되는데 `build(batch_input_shape)`을 호출해서 model을 만들때나 training/evaluation method (such as `.fit` or `.evaluate`)에 대한 첫번째 호출되는 때이다. 이것을 delayed-build pattern이라고 부른다.

따라서 이제는 `model` object를 생성하고 `build()`를 호출해서 만들어보자.(weights와 biases를 초기화)

In [ ]:
# model 객체 생성(Create a model object)
subclassed_model = Network(10)

# model 만들기(Build the model, i.e. initialize the model's weights and biases)
subclassed_model.build((None, 28, 28, 1))

subclassed_model.summary()

Remember that `None` is used to indicate that any integer may be expected. So, we use `None` to indicate batches of any size are acceptable. 

While model subclassing offers flexibility, it comes at a cost of greater complexity and more opportunities for
user errors. So, we recommend, to always use the simplest tool for the job. 

### Your Turn to Build a Neural Network

<img src="assets/mlp_mnist.png" width=600px>

> **Exercise:** Use the subclassing method to create a network with 784 input units, a hidden layer with 128 units and a ReLU activation, then a hidden layer with 64 units and a ReLU activation, and finally an output layer with 10 units and a softmax activation function. You can use a ReLU activation function by setting `activation = 'relu'`. After you create your model, create a model object and build it.

In [ ]:
## Solution

my_model_2 = 

my_model_2.summary()

## Looking at Weights and Biases of Subclassed Models

As before, we can get the weights an biases of each layer in our subclassed models. In this case, we can use the name we gave to each layer in the `__init__` method to get the weights and biases of a particular layer. For example, in the exercise we gave the first hidden layer the name `hidden_1`, so we can get the weights an biases from this layer by using:

In [ ]:
w1 = my_model_2.hidden_1.get_weights()[0]
b1 = my_model_2.hidden_1.get_weights()[1]

print('\n\u2022 Weights:\n', w)
print('\n\u2022 Biases:\n', b)
print('\nThis layer has a total of {:,} weights and {:,} biases'.format(w1.size, b1.size))

All the other methods we saw before, such as `.layers`, are also available for subclassed models, so feel free to use them.

## Making Predictions with Subclassed Models

Predictions are made in exactly the same way as before. So let's pass an image to our subclassed model and see what we get:

In [ ]:
for image_batch, label_batch in training_batches.take(1):
    ps = subclassed_model.predict(image_batch)
    first_image = image_batch.numpy().squeeze()[0]

fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
ax1.imshow(first_image, cmap = plt.cm.binary)
ax1.axis('off')
ax2.barh(np.arange(10), ps[0])
ax2.set_aspect(0.1)
ax2.set_yticks(np.arange(10))
ax2.set_yticklabels(np.arange(10))
ax2.set_title('Class Probability')
ax2.set_xlim(0, 1.1)
plt.tight_layout()

As before, we can see above, our model gives every digit roughly the same probability. This means our network has basically no idea what the digit in the image is. This is because we haven't trained our model yet, so all the weights are random!

In the next notebook, we'll see how we can train a neural network to accurately predict the numbers appearing in the MNIST images.

## Other Methods to Create Models

In [ ]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Flatten(input_shape = (28,28,1)))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
          
model.summary()

In [ ]:
layer_neurons = [512, 256, 128, 56, 28, 14]

model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape = (28,28,1)))

for neurons in layer_neurons:
    model.add(tf.keras.layers.Dense(neurons, activation='relu'))
            
model.add(tf.keras.layers.Dense(10, activation='softmax'))
          
model.summary()        

## Clearing the Graph

In order to avoid clutter from old models in the graph, we can use:

```python
tf.keras.backend.clear_session()
```

This command deletes the current `tf.keras` graph and creates a new one.

In [ ]:
tf.keras.backend.clear_session()

layer_neurons = [512, 256, 128, 56, 28, 14]

model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape = (28,28,1)))

for neurons in layer_neurons:
    model.add(tf.keras.layers.Dense(neurons, activation='relu'))
            
model.add(tf.keras.layers.Dense(10, activation='softmax'))
          
model.summary()    